---

### template

In [ ]:
# template 생성 예시
# template 생성의 장점은 Validation (유효성 검사)이 가능하다는 것임
# 예를 들어, 아래 코드에서 t.format()으로 실행하면, 오류가 발생함
# 또한, prompt template를 디스크에 저장하고, 다시 불러올 수 있음
t = PromptTemplate.from_template("What is the capital of {country}?")
t.format(country="France")

In [5]:
# 위에서 구현한 방식은 약간의 shortcut 방식임
# prompt template를 직접 구현할 수도 있음
t = PromptTemplate(
    template="What is the capital of {country}?"
    ,input_variables=["country"]
)
t.format(country="France")

'What is the capital of France?'

---

### FewShot Model

In [4]:
# from langchain.llms.openai import OpenAI # text-davinci-003 Model (사용 안 함)
from langchain.chat_models import ChatOpenAI # GPT-3.5-turbo Model (text-davinci-003의 1/10 가격)
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# llm = OpenAI()
chat = ChatOpenAI(
    temperature=0.1 # 낮을 수록 일관성 있음, 높을 수록 무작위성(창의성) 있음, default 0.7
    ,streaming=True # streaming=True로 설정하면, AI 출력이 나올 때마다 출력을 반환함, 모든 출력이 다 나올때까지 대기할 필요가 없음
    ,callbacks=[StreamingStdOutCallbackHandler()] # streaming=True로 설정하면, 출력을 받을 때마다 callback 함수를 호출함
)

별도의 예시 없이 chat.predict("What do you know about France?")를 즉시 실행하면 아래와 같은 답변이 나옴

France is a country located in Western Europe. It is known for its rich history, culture, and contributions to art, literature, and philosophy. Here are some key points about France:

1. Capital: The capital city of France is Paris, which is also its largest city.

2. Language: The official language is French, and it is spoken by the majority of the population.

3. Geography: France shares borders with several countries, including Belgium, Luxembourg, Germany, Switzerland, Italy, Spain, and Andorra. It also has coastlines along
.......

In [8]:
# FewShot Model은 모델에게 몇 개의 예시를 주고, 그 예시를 바탕으로 문장을 생성하도록 하는 방식임
# 이러한 예시를 주는 방식은 모델에게 직접 prompt로 주는 방식보다 더 정확한 문장을 생성하도록 할 수 있음
examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

In [7]:
example_template = """
    Human: {question}
    AI: {answer}
"""
example_prompt = PromptTemplate.from_template(example_template)

# 또는 아래와 같이 직접 구현할 수도 있음
# example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

In [11]:
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt
    ,examples=examples
    ,suffix="Human: What do you know about {country}?"
    ,input_variables=["country"]
)

# prompt.format(country="Germany")를 실행하면 AI에서 example에 마지막 질문을 더해 문장을 생성함
# chain을 생성해서 원하는 답안을 생성
chain = prompt | chat

chain.invoke(
    {
        "country":"Germany"
    }
)

AI: 
I know this:
Capital: Berlin
Language: German
Food: Bratwurst and Sauerkraut
Currency: Euro

AIMessageChunk(content='AI: \nI know this:\nCapital: Berlin\nLanguage: German\nFood: Bratwurst and Sauerkraut\nCurrency: Euro')

#### Prompt 만 생성했을 때

'\n    
Human: What do you know about France?\n    
AI: \n
Here is what I know:\n
Capital: Paris\n
Language: French\n
Food: Wine and Cheese\n
Currency: Euro\n\n\n\n\n    

Human: What do you know about Italy?\n    
AI: \n
I know this:\n
Capital: Rome\n
Language: Italian\n
Food: Pizza and Pasta\n
Currency: Euro\n\n\n\n\n    

Human: What do you know about Greece?\n    
AI: \n
I know this:\n
Capital: Athens\n
Language: Greek\n
Food: Souvlaki and Feta Cheese\n
Currency: Euro\n\n\n\n

Human: What do you know about Germany?'

#### chain을 사용하여 답변을 얻었을 때

'What is the capital of France?'
AI: 
I know this:
Capital: Berlin
Language: German
Food: Bratwurst and Sauerkraut
Currency: Euro

AIMessageChunk(content='AI: \nI know this:\nCapital: Berlin\nLanguage: German\nFood: Bratwurst and Sauerkraut\nCurrency: Euro')

---

### FromTemplate 대신 FromMessage 사용

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate( # suffix를 사용하지 않음
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Thailand"})


        I know this:
        Capital: Bangkok
        Language: Thai
        Food: Pad Thai and Tom Yum Soup
        Currency: Thai Baht
        

AIMessageChunk(content='\n        I know this:\n        Capital: Bangkok\n        Language: Thai\n        Food: Pad Thai and Tom Yum Soup\n        Currency: Thai Baht\n        ')

---

### ExampleSelector 적용

In [3]:
# from langchain.llms.openai import OpenAI # text-davinci-003 Model (사용 안 함)
from langchain.chat_models import ChatOpenAI # GPT-3.5-turbo Model (text-davinci-003의 1/10 가격)
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector

# llm = OpenAI()
chat = ChatOpenAI(
    temperature=0.1 # 낮을 수록 일관성 있음, 높을 수록 무작위성(창의성) 있음, default 0.7
    ,streaming=True # streaming=True로 설정하면, AI 출력이 나올 때마다 출력을 반환함, 모든 출력이 다 나올때까지 대기할 필요가 없음
    ,callbacks=[StreamingStdOutCallbackHandler()] # streaming=True로 설정하면, 출력을 받을 때마다 callback 함수를 호출함
)

examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

example_template = """
    Human: {question}
    AI: {answer}
"""
example_prompt = PromptTemplate.from_template(example_template)

example_selector = LengthBasedExampleSelector(
    examples=examples
    ,example_prompt=example_prompt
    ,max_length=80
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt
    # ,examples=examples <- example_selector를 사용하면 examples를 사용하지 않아도 됨
    ,example_selector=example_selector
    ,suffix="Human: What do you know about {country}?"
    ,input_variables=["country"]
)

# prompt.format(country="Germany")를 실행하면 AI에서 example에 마지막 질문을 더해 문장을 생성함
# chain을 생성해서 원하는 답안을 생성
chain = prompt | chat

chain.invoke(
    {
        "country":"Germany"
    }
)

'\n    Human: What do you know about France?\n    AI: \nHere is what I know:\nCapital: Paris\nLanguage: French\nFood: Wine and Cheese\nCurrency: Euro\n\n\n\n\n    Human: What do you know about Italy?\n    AI: \nI know this:\nCapital: Rome\nLanguage: Italian\nFood: Pizza and Pasta\nCurrency: Euro\n\n\n\nHuman: What do you know about France?'

---

### Random Example Selector

In [12]:
# from langchain.llms.openai import OpenAI # text-davinci-003 Model (사용 안 함)
from langchain.chat_models import ChatOpenAI # GPT-3.5-turbo Model (text-davinci-003의 1/10 가격)
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector



# llm = OpenAI()
chat = ChatOpenAI(
    temperature=0.1 # 낮을 수록 일관성 있음, 높을 수록 무작위성(창의성) 있음, default 0.7
    ,streaming=True # streaming=True로 설정하면, AI 출력이 나올 때마다 출력을 반환함, 모든 출력이 다 나올때까지 대기할 필요가 없음
    ,callbacks=[StreamingStdOutCallbackHandler()] # streaming=True로 설정하면, 출력을 받을 때마다 callback 함수를 호출함
)

examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples
    
    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = RandomExampleSelector(
    examples=examples
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt
    # ,examples=examples <- example_selector를 사용하면 examples를 사용하지 않아도 됨
    ,example_selector=example_selector
    ,suffix="Human: What do you know about {country}?"
    ,input_variables=["country"]
)

prompt.format(country="Germany")

'Human: What do you know about France?\nAI: \nHere is what I know:\nCapital: Paris\nLanguage: French\nFood: Wine and Cheese\nCurrency: Euro\n\n\nHuman: What do you know about Germany?'

---

### 저장소로부터 Prompt를 가져오는 방법

In [15]:
from langchain.chat_models import ChatOpenAI # GPT-3.5-turbo Model (text-davinci-003의 1/10 가격)
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

prompt_json = load_prompt("./prompt.json") # 해당 위치에서 prompt 형식을 불러옴
prompt_yaml = load_prompt("./prompt.yaml") # 해당 위치에서 prompt 형식을 불러옴

# llm = OpenAI()
chat = ChatOpenAI(
    temperature=0.1 # 낮을 수록 일관성 있음, 높을 수록 무작위성(창의성) 있음, default 0.7
    ,streaming=True # streaming=True로 설정하면, AI 출력이 나올 때마다 출력을 반환함, 모든 출력이 다 나올때까지 대기할 필요가 없음
    ,callbacks=[StreamingStdOutCallbackHandler()] # streaming=True로 설정하면, 출력을 받을 때마다 callback 함수를 호출함
)

print(prompt_json.format(country="Germany"))
print(prompt_yaml.format(country="Greece"))

What is the capital of Germany?
What is the capital of Greece?


---

### 여러 저장소에 흩어져 있는 prompt를 하나의 영역에 합치는 방법

In [16]:
from langchain.chat_models import ChatOpenAI # GPT-3.5-turbo Model (text-davinci-003의 1/10 가격)
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate

# llm = OpenAI()
chat = ChatOpenAI(
    temperature=0.1 # 낮을 수록 일관성 있음, 높을 수록 무작위성(창의성) 있음, default 0.7
    ,streaming=True # streaming=True로 설정하면, AI 출력이 나올 때마다 출력을 반환함, 모든 출력이 다 나올때까지 대기할 필요가 없음
    ,callbacks=[StreamingStdOutCallbackHandler()] # streaming=True로 설정하면, 출력을 받을 때마다 callback 함수를 호출함
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

Arrrrg! Me favorite food be a good ol' plate o' fish 'n chips! The taste o' crispy battered fish and salty chips be like a treasure for me taste buds, matey! Arg arg!!

AIMessageChunk(content="Arrrrg! Me favorite food be a good ol' plate o' fish 'n chips! The taste o' crispy battered fish and salty chips be like a treasure for me taste buds, matey! Arg arg!!")

---

### caching

In [19]:
from langchain.chat_models import ChatOpenAI # GPT-3.5-turbo Model (text-davinci-003의 1/10 가격)
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# cache를 사용하면, AI의 출력을 저장할 수 있음, 비슷한 질문에 대해서는 동일한 답변을 할 수 있음, 비용을 줄일 수 있음
set_llm_cache(InMemoryCache())

chat = ChatOpenAI(
    temperature=0.1 # 낮을 수록 일관성 있음, 높을 수록 무작위성(창의성) 있음, default 0.7
    # ,streaming=True # streaming=True로 설정하면, AI 출력이 나올 때마다 출력을 반환함, 모든 출력이 다 나올때까지 대기할 필요가 없음
    # ,callbacks=[StreamingStdOutCallbackHandler()] # streaming=True로 설정하면, 출력을 받을 때마다 callback 함수를 호출함
)

# check the time it takes to generate the output
import time
start_time = time.time()

print(chat.predict("How do you make italians pasta?"))

print("--- %s seconds ---" % (time.time() - start_time))

To make Italian pasta, follow these steps:

1. Boil water: Fill a large pot with water and bring it to a rolling boil. Add salt to the water to enhance the pasta's flavor.

2. Choose the pasta: Select the type of pasta you want to cook. Popular Italian pasta varieties include spaghetti, penne, fettuccine, linguine, or farfalle.

3. Add pasta to boiling water: Carefully add the pasta to the boiling water. Stir gently to prevent sticking.

4. Cook pasta: Follow the cooking time mentioned on the pasta package. Stir occasionally to ensure even cooking. Taste the pasta to check if it's cooked to your desired level of tenderness. Italians typically prefer their pasta al dente, which means it should be cooked but still firm to the bite.

5. Drain pasta: Once the pasta is cooked, carefully pour it into a colander or strainer to drain the water. Do not rinse the pasta, as the starch helps the sauce adhere better.

6. Prepare the sauce: While the pasta is cooking, you can prepare the sauce of yo

In [20]:
import time
start_time = time.time()

print(chat.predict("How do you make italians pasta?"))

print("--- %s seconds ---" % (time.time() - start_time))

To make Italian pasta, follow these steps:

1. Boil water: Fill a large pot with water and bring it to a rolling boil. Add salt to the water to enhance the pasta's flavor.

2. Choose the pasta: Select the type of pasta you want to cook. Popular Italian pasta varieties include spaghetti, penne, fettuccine, linguine, or farfalle.

3. Add pasta to boiling water: Carefully add the pasta to the boiling water. Stir gently to prevent sticking.

4. Cook pasta: Follow the cooking time mentioned on the pasta package. Stir occasionally to ensure even cooking. Taste the pasta to check if it's cooked to your desired level of tenderness. Italians typically prefer their pasta al dente, which means it should be cooked but still firm to the bite.

5. Drain pasta: Once the pasta is cooked, carefully pour it into a colander or strainer to drain the water. Do not rinse the pasta, as the starch helps the sauce adhere better.

6. Prepare the sauce: While the pasta is cooking, you can prepare the sauce of yo

### Debugger

In [21]:
from langchain.chat_models import ChatOpenAI # GPT-3.5-turbo Model (text-davinci-003의 1/10 가격)
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache

# cache를 사용하면, AI의 출력을 저장할 수 있음, 비슷한 질문에 대해서는 동일한 답변을 할 수 있음, 비용을 줄일 수 있음
set_llm_cache(InMemoryCache())

# debug를 사용하면 AI의 출력을 확인할 수 있음
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1 # 낮을 수록 일관성 있음, 높을 수록 무작위성(창의성) 있음, default 0.7
    # ,streaming=True # streaming=True로 설정하면, AI 출력이 나올 때마다 출력을 반환함, 모든 출력이 다 나올때까지 대기할 필요가 없음
    # ,callbacks=[StreamingStdOutCallbackHandler()] # streaming=True로 설정하면, 출력을 받을 때마다 callback 함수를 호출함
)

# check the time it takes to generate the output
import time
start_time = time.time()

print(chat.predict("How do you make italians pasta?"))

print("--- %s seconds ---" % (time.time() - start_time))

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italians pasta?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [15.62s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 1 pound (450 grams) of pasta (such as spaghetti, fettuccine, penne, or rigatoni)\n- Salt (for boiling water)\n- Water (for boiling pasta)\n- Olive oil (optional)\n- Sauce of your choice (such as marinara, Alfredo, pesto, or carbonara)\n- Grated Parmesan cheese (optional, for serving)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. Fill a large pot with water and bring it to a boil. Add salt to the boiling water (about 1-2 tablespoons per 4 liters of water) to season the pasta.\n\n2. Once the water is boiling, add the pasta to the pot. Stir occasionally to prevent sticking. Cook the pasta according to the package instructions or until it reaches your desired leve

### SQL을 사용한 DB Cache 저장

* langchain 문서 내 integreation으로 들어가면 수많은 Third-Party DB 제공 업체들에 대한 연결 방법이 나와 있음

In [24]:
from langchain.chat_models import ChatOpenAI # GPT-3.5-turbo Model (text-davinci-003의 1/10 가격)
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# cache를 사용하면, AI의 출력을 저장할 수 있음, 비슷한 질문에 대해서는 동일한 답변을 할 수 있음, 비용을 줄일 수 있음
# 특히 DB를 사용하면, 여러 사람이 동시에 사용할 수 있음, 또한 모델이 업데이트 되어도, cache를 사용하면 이전에 사용한 모델을 사용할 수 있음
set_llm_cache(SQLiteCache("cache.db"))

chat = ChatOpenAI(
    temperature=0.1 # 낮을 수록 일관성 있음, 높을 수록 무작위성(창의성) 있음, default 0.7
    # ,streaming=True # streaming=True로 설정하면, AI 출력이 나올 때마다 출력을 반환함, 모든 출력이 다 나올때까지 대기할 필요가 없음
    # ,callbacks=[StreamingStdOutCallbackHandler()] # streaming=True로 설정하면, 출력을 받을 때마다 callback 함수를 호출함
)

# check the time it takes to generate the output
import time
start_time = time.time()

print(chat.predict("How do you make italians pasta?"))

print("--- %s seconds ---" % (time.time() - start_time))

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italians pasta?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, follow these steps:\n\n1. Boil water: Fill a large pot with water and bring it to a rolling boil. Add salt to the water for seasoning.\n\n2. Choose the pasta: Select the type of pasta you want to cook. Popular Italian pasta varieties include spaghetti, penne, fettuccine, linguine, or farfalle.\n\n3. Add pasta to boiling water: Carefully add the pasta to the boiling water. Stir gently to prevent sticking.\n\n4. Cook pasta: Cook the pasta according to the package instructions or until it reaches your desired level of doneness. Italian pasta is typically cooked al dente, which means it should be firm to the bite.\n\n5. Prepare the sauce: While the pasta is cooking, you can prepare the sauce of your choice. Popular Italian pasta s

In [23]:
# check the time it takes to generate the output
import time
start_time = time.time()

print(chat.predict("How do you make italians pasta?"))

print("--- %s seconds ---" % (time.time() - start_time))

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italians pasta?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, follow these steps:\n\n1. Boil water: Fill a large pot with water and bring it to a rolling boil. Add salt to the water for seasoning.\n\n2. Choose the pasta: Select the type of pasta you want to cook. Popular Italian pasta varieties include spaghetti, penne, fettuccine, linguine, or farfalle.\n\n3. Add pasta to boiling water: Carefully add the pasta to the boiling water. Stir gently to prevent sticking.\n\n4. Cook pasta: Cook the pasta according to the package instructions or until it reaches your desired level of doneness. Italian pasta is typically cooked al dente, which means it should be firm to the bite.\n\n5. Prepare the sauce: While the pasta is cooking, you can prepare the sauce of your choice. Popular Italian pasta s

---

### 비용 관리

In [30]:
from langchain.chat_models import ChatOpenAI # GPT-3.5-turbo Model (text-davinci-003의 1/10 가격)
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1 # 낮을 수록 일관성 있음, 높을 수록 무작위성(창의성) 있음, default 0.7
)

with get_openai_callback() as usage: #callback 체크의 이름은 무관
    print(chat.predict("How do you make soju?")) # 처음 실행하는 코드에 대해서는 usage가 발생함
    print(chat.predict("How do you make italians pasta?")) # db에 cache가 있기 때문에 usage가 발생하지 않음
    print(usage)

To make soju, you will need the following ingredients and equipment:

Ingredients:
- 2 kilograms of rice
- 1 kilogram of nuruk (a Korean fermentation starter)
- 10 liters of water
- Distilled water
- Yeast (optional, for faster fermentation)

Equipment:
- Large pot or rice cooker
- Fermentation vessel (ceramic or glass jar)
- Airlock or plastic wrap with a rubber band
- Strainer or cheesecloth
- Bottles for storage

Here's a step-by-step guide on how to make soju:

1. Rinse the rice thoroughly to remove any impurities. Soak the rice in water for about 1-2 hours, then drain.

2. Cook the rice in a large pot or rice cooker with 10 liters of water until it becomes soft and fully cooked.

3. Once the rice is cooked, transfer it to a fermentation vessel while it's still hot. Allow it to cool down to around 30-35°C (86-95°F).

4. Add the nuruk to the cooked rice in the fermentation vessel. Mix it well with a sterilized spoon or your hands. Make sure the nuruk is evenly distributed.

5. Cover

---

### 직렬화 (Serialization)

In [35]:
from langchain.chat_models import ChatOpenAI # GPT-3.5-turbo Model (text-davinci-003의 1/10 가격)
from langchain.llms.openai import OpenAI

chat = OpenAI(
    temperature=0.1 # 낮을 수록 일관성 있음, 높을 수록 무작위성(창의성) 있음, default 0.7
    , max_tokens=450 # max_tokens를 설정하면, AI의 출력을 제한할 수 있음
    , model = "gpt-3.5-turbo-16k"
)

chat.save("model.json") # 모델을 저장함

In [38]:
from langchain.chat_models import ChatOpenAI # GPT-3.5-turbo Model (text-davinci-003의 1/10 가격)
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

chat = load_llm("model.json") # 모델을 불러옴

c:\Users\Public\Downloads\FullStackGPT\.venv\Lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\Public\Downloads\FullStackGPT\.venv\Lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
